In [ ]:
import os
import shutil
import cv2
import random
from tqdm import tqdm

fp = r"testing\100-test-videos"
vids = os.listdir(fp)
vid_paths = [os.path.join(fp, vid) for vid in vids]

out_dir = r"testing\assets\100-test-clips"

for p, vid in zip(vid_paths, vids):
    # select random starting point [0-(n-600)]
    # open writer
    # write 600 frames
    # save

    cap = cv2.VideoCapture(p)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) - 600)
    fps = cap.get(cv2.CAP_PROP_FPS)
    size = (int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),
            int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)))
    fourcc = cv2.VideoWriter_fourcc(*'MPEG')

    start_frame = random.randint(0, frame_count)

    vid_out_path = os.path.join(out_dir, vid).replace('mp4', '.avi')
    height, width, fps = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)), int(cap.get(
        cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(
            cv2.CAP_PROP_FPS))
    writer = cv2.VideoWriter(
        vid_out_path, cv2.VideoWriter_fourcc(*'MPEG'), fps, (width, height))
    

    for i in tqdm(range(0, frame_count)):
        ret, frame = cap.read()
        if not ret:
            break
        if i >= start_frame and i < start_frame + 600:
            writer.write(frame)
    
    writer.release()
    cap.release()

# Isolation Testing for TR-ROI Detection 
****

In [ ]:
from pipeline import *
from viz import *

In [ ]:
import cv2

out_dir = r"testing\assets\100-test-clips-results"
for fp, vid in zip(vid_paths, os.listdir(dir_path)):
    viz_path = os.path.join(out_dir, vid)
    roi = extract_roi_from_video(fp)
    visualize_roi(fp, viz_path, roi)

In [ ]:
dir_path = r"C:\Users\Levi\Desktop\contextualized-shot-quality-estimation\testing\test-batches\A-64-T-clips\hard"
dp = r"C:\Users\Levi\Desktop\contextualized-shot-quality-estimation\testing\test-batches\A-64-T-clips\hard\data"
vp = r"C:\Users\Levi\Desktop\contextualized-shot-quality-estimation\testing\test-batches\A-64-T-clips\hard\viz"


process_dir(dir_path, dp, vp)

In [4]:
import os
import cv2

finetunning_videos_dir = r"C:\Users\Levi\Desktop\roi-fine-tuning"
images_dir = r"C:\Users\Levi\Desktop\roi-finetuning-dataset\train\images"
vids = os.listdir(finetunning_videos_dir)

for vid in vids:
    vid_path = os.path.join(finetunning_videos_dir, vid)
    images_subdir = os.path.join(images_dir, vid)
    images_subdir = images_subdir.lower().replace('.avi', '')
    os.makedirs(images_subdir)
    cap = cv2.VideoCapture(vid_path)
    for i in range(600):
        ret, frame = cap.read()
        if not ret:
            break
        pad = (6 - len(str(i))) * '0'
        frame_name = "frame_" + pad + str(i) + ".png"
        frame_path = os.path.join(images_subdir, frame_name)
        cv2.imwrite(frame_path, frame)


# 971452_76124_southwestern university (tx)_75089_austin college_period1
# 971452_76124_southwestern university (tx)_75089_austin college_period1



